### 유튜브 api

In [1]:
# !pip install scrapetube
# !pip install google-api-python-client

In [2]:
import pandas as pd
import datetime
import scrapetube
from googleapiclient.discovery import build

import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
class YoutubeAPI:
    def __init__ (self):
        self.api_key = 'api_key'
        self.youtube = build('youtube', 'v3', developerKey=self.api_key)
        self.channel_id = 'UChlv4GSd7OQl3js-jkLOnFA' # 삼프로TV

        self.get_video_ids()
        
    def get_video_ids(self):
        videos = scrapetube.get_channel(self.channel_id)
        
        video_ids = []
        for video in videos:
            video_ids.append(video['videoId'])
        
        (date, date_range) = self.get_date_input()
        self.get_video_infos(video_ids, date, date_range)
        
    def get_date_input(self):
        date = input("날짜를 입력해 주세요 (예. 2022-08-04): ")
        date_date = datetime.datetime.strptime(date, "%Y-%m-%d")
        today = datetime.datetime.today()
        date_diff = (today - date_date).days
        date_range = (date_diff)
        
        return (date, date_range)
    
    def get_video_infos(self, video_ids, date, date_range):
        video_infos = []
        
        for i in range(date_range + 1):
            start = i * 50
            end = (i + 1) * 50
            
            video_request = self.youtube.videos().list(
                part='snippet',
                id=','.join(video_ids[start:end]))
            
            video_response = video_request.execute()
            
            for item in video_response['items']:
                title = item['snippet']['title']
                if ('글로벌 이슈체크' in title) or ('글로벌 마켓브리핑' in title) or ('직장인 vlog' in title):
                    continue
                if date in item['snippet']['publishedAt'].split()[0]:
                    video_infos.append([item['snippet']['title'], item['snippet']['publishedAt'], item['id']])
    
        df_ids = pd.DataFrame(video_infos, columns=['title', 'video_date', 'id'])
            
        self.get_comments(date, df_ids)
            
    def get_comments(self, date, df_ids):
        comments = []
        
        for video_id in df_ids['id']:
            api_obj = build('youtube', 'v3', developerKey=self.api_key)
            response = api_obj.commentThreads().list(part='snippet', videoId=video_id, maxResults=100).execute()
            
            while response:
                for item in response['items']:
                    comment = item['snippet']['topLevelComment']['snippet']
                    if date in comment['publishedAt'].split()[0]:
                        comments.append([video_id, comment['textDisplay'], comment['authorDisplayName'], comment['publishedAt'], comment['likeCount']])
            
                if 'nextPageToken' in response:
                    response = api_obj.commentThreads().list(part='snippet', videoId=video_id, pageToken=response['nextPageToken'], maxResults=100).execute()
                else:
                    break
                
        df_comments = pd.DataFrame(comments, columns=['id', 'comment', 'author', 'comment_date', 'num_likes'])
        
        df = pd.merge(df_comments, df_ids, on='id', how='outer')

        print(df)
        
        # df.to_csv(f'sampro_{date}.csv', index=False)
        
        # self.update_db(df)
        
    def update_db(self, df):
        pass
    
    def execute_daily(self):
        pass

In [ ]:
y_api = YoutubeAPI()